In [1]:
import pandas as pd
from rapidfuzz import process, fuzz
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI
from fuzzywuzzy import fuzz, process

# Load the .env file
load_dotenv()

c:\Users\iamth\Desktop\Sasta\app\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


True

In [2]:
# Ensure the "jumbo" directory exists
if not os.path.exists('albert_heijn'):
    os.makedirs('albert_heijn')

In [3]:
categories = {
  "Aardappel, groente, fruit": "https://www.ah.nl/producten/aardappel-groente-fruit",
  "Salades, pizza, maaltijden": "https://www.ah.nl/producten/salades-pizza-maaltijden",
  "Vlees, vis": "https://www.ah.nl/producten/vlees-vis",
  "Vegetarisch, vegan en plantaardig": "https://www.ah.nl/producten/vegetarisch-vegan-en-plantaardig",
  "Kaas, vleeswaren, tapas": "https://www.ah.nl/producten/kaas-vleeswaren-tapas",
  "Zuivel, eieren, boter": "https://www.ah.nl/producten/zuivel-eieren-boter",
  "Bakkerij": "https://www.ah.nl/producten/bakkerij",
  "Ontbijtgranen en beleg": "https://www.ah.nl/producten/ontbijtgranen-en-beleg",
  "Chips, noten, toast, popcorn": "https://www.ah.nl/producten/chips-noten-toast-popcorn",
  "Snoep, chocolade, koek": "https://www.ah.nl/producten/snoep-chocolade-koek",
  "Tussendoortjes": "https://www.ah.nl/producten/tussendoortjes",
  "Koffie, thee": "https://www.ah.nl/producten/koffie-thee",
  "Frisdrank, sappen, siropen, water": "https://www.ah.nl/producten/frisdrank-sappen-siropen-water",
  "Wijn en bubbels": "https://www.ah.nl/producten/wijn-en-bubbels",
  "Bier en aperitieven": "https://www.ah.nl/producten/bier-en-aperitieven",
  "Pasta, rijst en wereldkeuken": "https://www.ah.nl/producten/pasta-rijst-en-wereldkeuken",
  "Soepen, sauzen, kruiden, olie": "https://www.ah.nl/producten/soepen-sauzen-kruiden-olie",
  "Diepvries": "https://www.ah.nl/producten/diepvries",
  "Drogisterij": "https://www.ah.nl/producten/drogisterij",
  "Gezondheid, sport": "https://www.ah.nl/producten/gezondheid-sport",
  "Baby en kind": "https://www.ah.nl/producten/baby-en-kind",
  "Huishouden": "https://www.ah.nl/producten/huishouden",
  "Huisdier": "https://www.ah.nl/producten/huisdier",
  "Koken, tafelen, vrije tijd": "https://www.ah.nl/producten/koken-tafelen-vrije-tijd"
}

In [6]:
# Use this if the database is already created else run the next code block
df_products = pd.read_csv("./Albert Heijn/ah_products.csv")

In [8]:
df_products.head()

,Category,Product Name,Product Link,Price,Quantity,Product Description,Product ID
0,"Aardappel, groente, fruit",Bieze Rauwkost coleslawAdvertentie,https://www.ah.nl/producten/product/wi203226/b...,1.99,250 g,Bieze Rauwkost coleslawAdvertentie 250 g,1
1,"Aardappel, groente, fruit",Bieze Rauwkost farmerAdvertentie,https://www.ah.nl/producten/product/wi203227/b...,1.99,250 g,Bieze Rauwkost farmerAdvertentie 250 g,2
2,"Aardappel, groente, fruit",AH Komkommer,https://www.ah.nl/producten/product/wi54074/ah...,0.95,per stuk,AH Komkommer per stuk,3
3,"Aardappel, groente, fruit",AH Bananen tros,https://www.ah.nl/producten/product/wi197393/a...,1.55,Tros,AH Bananen tros Tros,4
4,"Aardappel, groente, fruit",AH Nederlandse aardbeien,https://www.ah.nl/producten/product/wi491168/a...,3.69,400 g,AH Nederlandse aardbeien 400 g,5


In [45]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# # Headers to mimic a browser request
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
# }

# page_number = 0

# # Flag to control the loop
# continue_scraping = True

# # Initialize an empty list to collect data
# data = []

# while continue_scraping:
#     # Loop through the dictionary and print each URL
#     for category_name, category_link in categories.items():
#         # Construct the URL for the current page
#         url = f"{category_link}?page={page_number}"
#         response = requests.get(url, headers=headers)
#         soup = BeautifulSoup(response.content, 'lxml')

#         product_containers = soup.find_all('article', class_='product-card-portrait_root__ZiRpZ product-grid-lane_gridItems__BBa4h')

#         if not product_containers:
#             print(f"No more products found on page {page_number} for category {category_name}. Finished scraping category.")
#             # Set the flag to False to break out of both loops
#             continue_scraping = False
#             break
        
#         else:
#             # Iterate over each product container and extract the required information
#             for product in product_containers:
#                 # Extract the product name
#                 product_name = product.find('div', class_='product-card-portrait_content__DQ9nP').get_text(strip=True)
#                 product_link = product.find('a', class_='link_root__EqRHd product-card-portrait_link__5VsEK').get('href')
#                 product_price_int = product.find('span', class_='price-amount_integer__+e2XO').get_text(strip=True)
#                 product_price_decimal = product.find('span', class_='price-amount_fractional__kjJ7u').get_text(strip=True)
#                 product_quantity = product.find("span", class_='price_unitSize__Hk6E4').get_text(strip=True)

#                 # Construct the full product link
#                 full_product_link = f"https://www.ah.nl{product_link}"

#                 # Combine integer and decimal parts of the price
#                 product_price = f"{product_price_int}.{product_price_decimal}"

#                 # Append the data to the list
#                 data.append({
#                     'Category': category_name,
#                     'Product Name': product_name,
#                     'Product Link': full_product_link,
#                     'Price': product_price,
#                     'Quantity': product_quantity
#                 })
    
#     page_number += 1

In [46]:
# # Create a DataFrame from the list of data
# df_products = pd.DataFrame(data)
# df_products['Product Description'] = df_products['Product Name'] + " " + df_products['Quantity']
# df_products['Product ID'] = range(1, len(df_products) + 1)

# # Optionally, save the DataFrame to a CSV file
# df_products.to_csv('ah_products.csv', index=False)

In [47]:
# df_products.head()

In [18]:
df_my_grocery_list = pd.read_csv('my_grocery_list.csv')
df_my_grocery_list['Product Description'] = df_my_grocery_list['Item Name'] + " " + df_my_grocery_list['Amount'] + " " + df_my_grocery_list['Quantity']
df_my_grocery_list.head()

,Item Name,Amount,Quantity,Product Description
0,Noten Mix gezouten,200g,1 pack,Noten Mix gezouten 200g 1 pack
1,Wasa Volkoren,275g,1 pack,Wasa Volkoren 275g 1 pack
2,Mager rundergehakt,500g,1 pack,Mager rundergehakt 500g 1 pack
3,Bananen,-,6 pieces,Bananen - 6 pieces
4,Maza Hoemoes,250g,1 tub,Maza Hoemoes 250g 1 tub


In [20]:
# Initialize OpenAI client with your API key
client =  OpenAI()

# Function to use ChatGPT for categorizing items
def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# Use the keys from the dictionary as category names
category_names = list(categories.keys())

In [21]:
category_names

['Aardappel, groente, fruit',
 'Salades, pizza, maaltijden',
 'Vlees, vis',
 'Vegetarisch, vegan en plantaardig',
 'Kaas, vleeswaren, tapas',
 'Zuivel, eieren, boter',
 'Bakkerij',
 'Ontbijtgranen en beleg',
 'Chips, noten, toast, popcorn',
 'Snoep, chocolade, koek',
 'Tussendoortjes',
 'Koffie, thee',
 'Frisdrank, sappen, siropen, water',
 'Wijn en bubbels',
 'Bier en aperitieven',
 'Pasta, rijst en wereldkeuken',
 'Soepen, sauzen, kruiden, olie',
 'Diepvries',
 'Drogisterij',
 'Gezondheid, sport',
 'Baby en kind',
 'Huishouden',
 'Huisdier',
 'Koken, tafelen, vrije tijd']

In [22]:
# Function to categorize item using ChatGPT
def categorize_item(item_name, category_names):
    prompt = f"Categorize the following grocery item: '{item_name}' \
            into one of these categories: :\n- " + "\n- ".join(category_names) + ".\n \
            Strictly stick to one of these categories and return the category name. Don't return anything else.\
            For example: \
            Blauwe Bessen should return Aardappel, groente, fruit and not Fruit \
            Spinazie should return Aardappel, groente, fruit and not Groente and so on"
    
    print(prompt)
    try:
        return chat_gpt(prompt)
    except Exception as e:
        print(f"Error categorizing item '{item_name}': {e}")
        return None

# Add a 'Category' column to the DataFrame using the item names
df_my_grocery_list['Category'] = df_my_grocery_list['Item Name'].apply(lambda x: categorize_item(x, category_names))

Categorize the following grocery item: 'Noten Mix gezouten'             into one of these categories: :
- Aardappel, groente, fruit
- Salades, pizza, maaltijden
- Vlees, vis
- Vegetarisch, vegan en plantaardig
- Kaas, vleeswaren, tapas
- Zuivel, eieren, boter
- Bakkerij
- Ontbijtgranen en beleg
- Chips, noten, toast, popcorn
- Snoep, chocolade, koek
- Tussendoortjes
- Koffie, thee
- Frisdrank, sappen, siropen, water
- Wijn en bubbels
- Bier en aperitieven
- Pasta, rijst en wereldkeuken
- Soepen, sauzen, kruiden, olie
- Diepvries
- Drogisterij
- Gezondheid, sport
- Baby en kind
- Huishouden
- Huisdier
- Koken, tafelen, vrije tijd.
             Strictly stick to one of these categories and return the category name. Don't return anything else.            For example:             Blauwe Bessen should return Aardappel, groente, fruit and not Fruit             Spinazie should return Aardappel, groente, fruit and not Groente and so on
Categorize the following grocery item: 'Wasa Volkoren'    

In [23]:
print(df_my_grocery_list.to_string())

                   Item Name   Amount  Quantity                   Product Description                       Category
0         Noten Mix gezouten     200g    1 pack        Noten Mix gezouten 200g 1 pack   Chips, noten, toast, popcorn
1              Wasa Volkoren     275g    1 pack             Wasa Volkoren 275g 1 pack                       Bakkerij
2         Mager rundergehakt     500g    1 pack        Mager rundergehakt 500g 1 pack                     Vlees, vis
3                    Bananen        -  6 pieces                    Bananen - 6 pieces      Aardappel, groente, fruit
4               Maza Hoemoes     250g     1 tub               Maza Hoemoes 250g 1 tub     Salades, pizza, maaltijden
5               Magere kwark     500g    2 tubs              Magere kwark 500g 2 tubs          Zuivel, eieren, boter
6   Scharrel kipfiletblokjes     400g    1 pack  Scharrel kipfiletblokjes 400g 1 pack                     Vlees, vis
7              Volkorenbrood     800g    1 loaf             Volk

In [24]:
ls_unique_categories = df_my_grocery_list['Category'].unique().tolist()
df_selected_products = df_products[df_products['Category'].isin(ls_unique_categories)]

# df_selected_products = df_products

In [25]:
df_selected_products.head()

,Category,Product Name,Product Link,Price,Quantity,Product Description,Product ID
0,"Aardappel, groente, fruit",Bieze Rauwkost coleslawAdvertentie,https://www.ah.nl/producten/product/wi203226/b...,1.99,250 g,Bieze Rauwkost coleslawAdvertentie 250 g,1
1,"Aardappel, groente, fruit",Bieze Rauwkost farmerAdvertentie,https://www.ah.nl/producten/product/wi203227/b...,1.99,250 g,Bieze Rauwkost farmerAdvertentie 250 g,2
2,"Aardappel, groente, fruit",AH Komkommer,https://www.ah.nl/producten/product/wi54074/ah...,0.95,per stuk,AH Komkommer per stuk,3
3,"Aardappel, groente, fruit",AH Bananen tros,https://www.ah.nl/producten/product/wi197393/a...,1.55,Tros,AH Bananen tros Tros,4
4,"Aardappel, groente, fruit",AH Nederlandse aardbeien,https://www.ah.nl/producten/product/wi491168/a...,3.69,400 g,AH Nederlandse aardbeien 400 g,5


In [27]:
def find_closest_products(df_my_grocery_list, df_selected_products, top_n=10, scorer=fuzz.token_set_ratio):
    # Create an empty list to collect results
    results = []

    # Iterate through each row in df_my_grocery_list
    for index, row in df_my_grocery_list.iterrows():
        my_product_description = row['Product Description']
        category = row['Category']

        # Filter df_selected_products by the category of the current item
        filtered_products = df_selected_products[df_selected_products['Category'] == category]

        # Perform fuzzy matching using the specified scoring function on 'Product Description'
        matches = process.extract(my_product_description, filtered_products['Product Description'], scorer=scorer, limit=top_n)

        if not matches:  # If no matches are found
            # Append the current item from df_my_grocery_list with None for matched product details
            results.append({
                'Item Name (My List)': row['Item Name'],
                'Amount (My List)': row['Amount'],
                'Quantity (My List)': row['Quantity'],
                'Product Description (My List)': my_product_description,
                'Category (My List)': category,
                'Product Description (Selected Product)': None,
                'Product ID (Selected Product)': None,
                'Product Name (Selected Product)': None,
                'Product Link (Selected Product)': None,
                'Price (Selected Product)': None,
                'Quantity Available (Selected Product)': None,
                'Confidence Score': None
            })
        else:
            # Extract matched Product Descriptions, IDs, Prices, Quantities, and Confidence Scores
            for match in matches:
                match_product_description = match[0]
                confidence_score = match[1]  # Extract the confidence score from the match tuple
                match_product_row = filtered_products[filtered_products['Product Description'] == match_product_description].iloc[0]
                match_product_id = match_product_row['Product ID']
                match_price = match_product_row['Price']
                match_quantity_available = match_product_row['Quantity']
                match_product_name = match_product_row['Product Name']
                match_product_link = match_product_row['Product Link']

                # Append all information from df_my_grocery_list and matched data to results list
                results.append({
                    'Item Name (My List)': row['Item Name'],
                    'Amount (My List)': row['Amount'],
                    'Quantity (My List)': row['Quantity'],
                    'Product Description (My List)': my_product_description,
                    'Category (My List)': category,
                    'Product Description (Selected Product)': match_product_description,
                    'Product ID (Selected Product)': match_product_id,
                    'Product Name (Selected Product)': match_product_name,
                    'Product Link (Selected Product)': match_product_link,
                    'Price (Selected Product)': match_price,
                    'Quantity Available (Selected Product)': match_quantity_available,
                    'Confidence Score': confidence_score
                })

    # Convert results list to DataFrame
    df_closest_products = pd.DataFrame(results)

    return df_closest_products

# Example usage:
df_closest_products = find_closest_products(df_my_grocery_list, df_selected_products)

In [28]:
print(df_closest_products.to_string())

df_closest_products = df_closest_products[['Item Name (My List)','Amount (My List)', 'Quantity (My List)', 
                                          'Product Name (Selected Product)', 'Category (My List)', 'Product Link (Selected Product)',
                                            'Price (Selected Product)', 'Quantity Available (Selected Product)',  'Confidence Score']]

          Item Name (My List) Amount (My List) Quantity (My List)         Product Description (My List)             Category (My List)                 Product Description (Selected Product)  Product ID (Selected Product)                 Product Name (Selected Product)                                                              Product Link (Selected Product)  Price (Selected Product) Quantity Available (Selected Product)  Confidence Score
0          Noten Mix gezouten             200g             1 pack        Noten Mix gezouten 200g 1 pack   Chips, noten, toast, popcorn                             AH Notenmix gezouten 150 g                           6593                            AH Notenmix gezouten                            https://www.ah.nl/producten/product/wi217662/ah-notenmix-gezouten                      2.95                                 150 g         67.857143
1          Noten Mix gezouten             200g             1 pack        Noten Mix gezouten 200g 1 pack   Chips,

In [29]:
# Remove duplicates from df_closest_products by keeping the row with the highest confidence score for each item in df_my_grocery_list
df_closest_products_unique = df_closest_products.sort_values(by=['Item Name (My List)', 'Confidence Score'], ascending=[True, False])

# Drop duplicates based on 'Item Name (My List)' to keep the best match
df_closest_products_unique = df_closest_products_unique.drop_duplicates(subset=['Item Name (My List)'], keep='first')

print(df_closest_products_unique)

          Item Name (My List) Amount (My List) Quantity (My List)  \
80                  Aardbeien             250g             1 pack   
230               Afwasmiddel            500ml           1 bottle   
240            Aluminiumfolie              10m             1 roll   
140               Amandelmelk          1 liter           1 carton   
160                   Avocado                -           2 pieces   
30                    Bananen                -           6 pieces   
290                Batterijen                -             1 pack   
170             Blauwe bessen             125g             1 pack   
120             Geraspte kaas             150g             1 pack   
180           Griekse yoghurt              1kg              1 tub   
150                 Havermout             500g             1 pack   
220              Keukenpapier                -            2 rolls   
20         Mager rundergehakt             500g             1 pack   
50               Magere kwark     

In [30]:
df_closest_products_unique.to_csv('ah_output.csv', index=False)